In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths
biomed_path = "/content/drive/MyDrive/BIOMED"
ham10000_path = os.path.join(biomed_path, "HAM10000")

# Load metadata
metadata = pd.read_csv(os.path.join(biomed_path, "HAM10000_metadata.csv"))

# Update the image_id column to include the full path to the images
metadata["image_id"] = metadata.apply(lambda row: os.path.join(biomed_path, "HAM10000_images_part_1", row["image_id"] + ".jpg") if row["image_id"][0] == 'I' else os.path.join(biomed_path, "HAM10000_images_part_2", row["image_id"] + ".jpg"), axis=1)

# Split the dataset into training, validation, and test sets
train_df, test_df = train_test_split(metadata, test_size=0.2, random_state=42, stratify=metadata["dx"])
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42, stratify=train_df["dx"])

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

image_size = 299
batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_id",
    y_col="dx",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col="image_id",
    y_col="dx",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)

# Load Xception model
base_model = Xception(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(7, activation="softmax", kernel_regularizer=regularizers.l2(0.01))(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
num_epochs = 25
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    epochs=num_epochs,
)

# Save the trained model
model.save("xception_ham10000.h5")
